In [ ]:
from functions.keygen import generate_rsa_keys
from functions.Utils import os2ip, i2osp 
import hashlib, json, base64

# Assinatura

Começando com a criação do arquivo que será assinado e obtendo as chaves publica e privada:

In [38]:
with open("documento.txt", "w") as f:
    f.write("A principal vantagem do OAEP e transformar o RSA de um algoritmo deterministico" \
    " em um probabilistico, dificultando ataques por analise de padroes. Alem disso, ele incorpora" \
    " verificacoes internas que detectam alteracoes ou corrupcoes nos dados antes da decifracao. ")

public_key_signer, private_key_signer = generate_rsa_keys(bits=1024)

file_to_sign = "documento.txt" 

print()
print(f'n: {public_key_signer[0]}\ne: {public_key_signer[1]}')
print()
print(f'n: {private_key_signer[0]}\nd: {private_key_signer[1]}')

Gerando p (primo de 1024 bits)...
p gerado: 97343072851515034258703743432763162845054019495014869828040938470587314449290816776503197112894877200442098056498015816219546452251992192879041564729458627547275278602252255104689018520969222600395420484806827027257786896515431228434580276900727615187531020141111396193119023834861363662161642900780002109143
Gerando q (primo de 1024 bits)...
q gerado: 162597836603072150766337720304530496953572059568951753152467527144485365174335445397583067343686392686416543650332963454020797957402410895266577527727800433317011427359528957912234729653226724892988420918761938348601993679887026000310388205769851024703184648902018098073508891132356462338174827894770536696163

n: 1582777305395159019869856385145448879416280702653651828781071209811997693584047966774658201505232609152671879935104617223788939551686949009681681629169715152101783252146367151995041238316525829173557325121323462685911875903539241225050554614139823122944623334444974011764051434993832509190

Com o arquivo e as chaves já em mãos é necessário uma função de hash par resumir a mensagem no arquivo:

In [25]:
def sha3_hash(filepath, hash_algo='sha3-256'):
    """
    Calcula o hash SHA-3 de um arquivo.
    filepath: Caminho para o arquivo a ser hasheado.
    hash_algo: O algoritmo SHA-3 a ser usado (ex: 'sha3-256', 'sha3-512').
    
    Retorna os bytes do hash (digest).
    """
    try:
        hasher = hashlib.new(hash_algo)
    except ValueError:
        raise ValueError(f"Algoritmo de hash '{hash_algo}' não suportado ou inválido.")

    with open(filepath, 'rb') as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    
    return hasher.digest()

In [39]:
message_hash_bytes = sha3_hash(file_to_sign)
print(f"Hash do documento ({len(message_hash_bytes)} bytes): {message_hash_bytes.hex()}")

Hash do documento (32 bytes): 82f3d3ad5d3ba567ec649f48aef2fea3cafa911f92dd5fbccbbf3d61034aa2f3


Em seguida é necessário assinar esse documento:

In [27]:
def sign_message_hash(message_hash_bytes, private_key):
   
    n, d = private_key
    k = (n.bit_length() + 7) // 8 

    # Converte o hash da mensagem para um inteiro.
    hash_int = os2ip(message_hash_bytes)

    # A assinatura RSA
    signature_int = pow(hash_int, d, n)
    
    # Converte o inteiro da assinatura de volta para uma string de octetos (bytes).
    # O comprimento dessa string de octetos deve ser igual ao comprimento do módulo n (k).
    # Isso garante que a assinatura tenha o tamanho correto para o bloco RSA.
    signature_bytes = i2osp(signature_int, k)
    
    return signature_bytes

In [28]:
signature_bytes = sign_message_hash(message_hash_bytes, private_key_signer)
print(f"Assinatura ({len(signature_bytes)} bytes): {signature_bytes.hex()}")

Assinatura (256 bytes): 28fa316401772d988ec3792b2b61142418554a7c1c8183921ad944ffd513802d5229afab4b2835fa2e66442f4a57af794f46262e9d68ce87b564be9504dd00f8f14c0fe00f34cd38cefda33167a9ec5877f46db7007c2101bef41e1b1378f22f7f1fbd7c4165364d8628918212135c08b11cedcaef7d40babc551ca018cc08f0f279f8918b1ee3c1b351a56741cdca256c420485063635fbf05dcb4070d897ce78f69eb280a46e539ce4bdaaa16915755aeb7ac3852fdba941f760c2fdc8723ae0ace4c79f198f954c4cdb4a7305d6488b341d6461cd846091bb01256661f1d599c2879c89f641aefb5bd09c10c66ab235a04843e686287b2533b5f65608e706


Por ultimo é preciso formatar o arquivo da assinatura. Para isso a assinatura, o algoritmo de hash usado e a chave publica que valida a assinatura são organizados em um arquivo json, prontos para serem salvos em um arquivo .sig

In [29]:
def format_signature_output(signature_bytes, public_key_signer, hash_algorithm_name):
    
    n_signer, e_signer = public_key_signer

    n_str = str(n_signer)
    e_str = str(e_signer)
    
    signature_b64 = base64.b64encode(signature_bytes).decode('utf-8')

    signature_data = {
        "hash_algorithm": hash_algorithm_name,
        "public_key_n": n_str,
        "public_key_e": e_str,
        "signature": signature_b64
    }

    json_string = json.dumps(signature_data, indent=2)
    json_bytes = json_string.encode('utf-8')
    final_b64_string = base64.b64encode(json_bytes).decode('utf-8')

    return final_b64_string

In [30]:
signed_document_content = format_signature_output(signature_bytes, public_key_signer, "SHA3-256")
print(f"\nConteúdo do documento assinado (Base64):\n{signed_document_content}")


with open(file_to_sign + ".sig", "w") as f:
    f.write(signed_document_content)


Conteúdo do documento assinado (Base64):
ewogICJoYXNoX2FsZ29yaXRobSI6ICJTSEEzLTI1NiIsCiAgInB1YmxpY19rZXlfbiI6ICIxNjYyODk0NjgzMDIzMDQ4MzE1NDc4NzYyNTEzNjI5MzUwNzc3NjExMjY3MjIwOTI5ODA5NjU1NjM4Njc3NTk1MzUxMjk4NTU2MDI2NjcxMDk0NzYwODAwMjE4MzIyODA2MzI4NDEyNjQ0ODU5ODY3MTAzMzYyMjkyMTYyNzE1MTUzNDU4NzMyNDg4MDM1NjM0Nzg4OTA5MjE4MDU2OTkzMzQ0MTc1NjIxMDIyNzY2MjYyNjk3OTcxMDUxODE0NjI3NjIyODgzMzA4NzU5NDMxMjc4NzY4NTE0OTMxMDY5NDk2NzA2NTY0MjAyMTE4ODUwOTM2NzI0NjYwNTcyMzkxNjQ0ODIzMDY1NDkxNzQzMTk5NDU4NTI2MjE3MDE5MDM5NDk0MjY4MTAwMTc3NzgzNzk0NDMwNTU3Mzg3NzkyMjYwNTIyMzIxMTI4NzU3NTcxOTE0ODU5NjQ1NzQ2MzgwMDM2MTUyMDk5NzQ4NTIxMDIzODYyMzE4MTc2Njk2MDQzMTgyOTE3NTU2NjU0MTA4MzIwNDAzOTU0MTc3NDY2OTA2OTY3NTM0MzEyNjc1MTA4Mzk5NTQ1NzMwOTUyNzcyOTQ3MTI1OTg5NDA0MTAwMDA3OTkyODU0MDQ4MDAwNDMzMzMzNjkwMjc0NTMyNTc4OTU2MTU0MTk0OTczMTc5NjUyMTc3MTcxODA2NzExMDY5OTA4MzYwNjYzMzgxMjI1NzE3NTE5MTk3MjA3NDIwMzkzMjAxNzAwOTg1NTcxODkwOTc2OTQ3ODgxMzQwMzg2ODMwMjkyODkyMTE2OTcyNzU4NTA1MDMwMyIsCiAgInB1YmxpY19rZXlfZSI6ICI2NTUzNyIsCiAgInNpZ25hdHVyZSI6ICJLUG

# Verificação

Agora para a verificação é preciso verificar o arquivo que foi assinado e calcular o hash dele. AO mesmo tempo a assinatura passa por um parser, para separar cada informação e o hash da assinatura é recuperado e descriptografado.

In [31]:
original_doc = "documento.txt"
signed_doc_file = "documento.txt.sig"

Primeiramente temos o parser, que reconstroi o json a partir do arquivo com a assinatura

In [32]:
def parse_signed_document(signed_file_path):
    
    with open(signed_file_path, 'r') as f:
        encoded_content = f.read()
    
    try:
        json_bytes = base64.b64decode(encoded_content)
        json_string = json_bytes.decode('utf-8')
        signature_data = json.loads(json_string)

        hash_algorithm = signature_data.get("hash_algorithm")
        
        public_key_n = int(signature_data.get("public_key_n"))
        public_key_e = int(signature_data.get("public_key_e"))
        public_key_signer = (public_key_n, public_key_e)
        
        signature_b64_from_json = signature_data.get("signature")
        
        if signature_b64_from_json is None:
            raise ValueError("Chave 'signature' não encontrada no arquivo JSON assinado.")

        signature_bytes = base64.b64decode(signature_b64_from_json)

        return {
            "hash_algorithm": hash_algorithm,
            "public_key_signer": public_key_signer,
            "signature_bytes": signature_bytes
        }

    except (base64.binascii.Error, json.JSONDecodeError, ValueError, TypeError) as e:
        raise ValueError(f"Erro ao analisar o arquivo de assinatura: {e}. O formato pode estar inválido.")

Em seguida é preciso descriptografar o hash

In [33]:
def decrypt_signature_hash(signature_bytes, public_key, expected_hash_byte_length):
  
    n, e = public_key
    k = (n.bit_length() + 7) // 8 

    signature_int = os2ip(signature_bytes)

    # Validação básica do valor da assinatura
    if not (0 <= signature_int < n):
        raise ValueError("Assinatura inválida: valor fora do intervalo do módulo [0, n-1].")


    decrypted_value_int = pow(signature_int, e, n)
    
    decrypted_full_bytes = i2osp(decrypted_value_int, k)
    
    # Se o hash original (ex: 32 bytes) foi assinado diretamente (sem padding complexo),
    # ele estará contido nos bytes decifrados, provavelmente no final, com zeros à esquerda.
    # Precisamos extrair apenas os bytes que correspondem ao hash.
    
    # Pega os últimos `expected_hash_byte_length` bytes.
    # Ex: se o hash é 32 bytes e o bloco é 128 bytes, pega os últimos 32 bytes.
    decrypted_hash_bytes = decrypted_full_bytes[-expected_hash_byte_length:]
    
    return decrypted_hash_bytes

Por ultimo fazemos a verificação, comparando o hash recuperado do arquivo .sig e o calculado a partir do original.

In [34]:
def verify_signature(original_file_path, signed_file_path):
    
       # Parsing do documento assinado e extração das informações
       signature_info = parse_signed_document(signed_file_path)
       
       signature_bytes = signature_info["signature_bytes"]
       public_key_signer = signature_info["public_key_signer"]
       hash_algorithm_name = signature_info["hash_algorithm"]


       # Determina o comprimento esperado do hash em bytes
       # Usando hashlib para obter o digest_size de forma confiável
       try:
           hasher_temp = hashlib.new(hash_algorithm_name)
           expected_hash_len_bytes = hasher_temp.digest_size
       except ValueError:
           raise ValueError(f"Algoritmo de hash '{hash_algorithm_name}' não reconhecido para determinar o tamanho do digest.")
       

       # Decifração da assinatura para obter o hash supostamente original
       decrypted_hash_bytes = decrypt_signature_hash(signature_bytes, public_key_signer, expected_hash_len_bytes)
       print(f"DEBUG: Decrypted hash (bytes): {decrypted_hash_bytes.hex()}")

       # Recalcular o hash do arquivo original
       actual_file_hash_bytes = sha3_hash(original_file_path, hash_algorithm_name)
       print(f"DEBUG: Actual file hash (bytes): {actual_file_hash_bytes.hex()}")

       
       if decrypted_hash_bytes == actual_file_hash_bytes:
           print(f"Assinatura válida para o arquivo '{original_file_path}'!")
           return True
       else:
           print(f"Assinatura inválida para o arquivo '{original_file_path}'. Os hashes não correspondem.")
           return False

Agora vamos testar a verificação:

In [35]:
is_valid = verify_signature(original_doc, signed_doc_file)

if is_valid:
    print("\nVerificação concluída com sucesso: Assinatura é autêntica.")
else:
    print("\nVerificação concluída: Assinatura não é autêntica ou foi adulterada.")

DEBUG: Decrypted hash (bytes): 1f24d41b4fcea277edabd8d5db9b21649bcc63c742d36f0318a290d454e434b9
DEBUG: Actual file hash (bytes): 1f24d41b4fcea277edabd8d5db9b21649bcc63c742d36f0318a290d454e434b9
Assinatura válida para o arquivo 'documento.txt'!

Verificação concluída com sucesso: Assinatura é autêntica.


Agora testaando para o arquivo modificado

In [36]:
with open("documento.txt", "a") as f:
    f.write(".")


is_valid_after_tamper = verify_signature(original_doc, signed_doc_file)

if not is_valid_after_tamper:
    print("\nAdulteração detectada com sucesso (como esperado).")

DEBUG: Decrypted hash (bytes): 1f24d41b4fcea277edabd8d5db9b21649bcc63c742d36f0318a290d454e434b9
DEBUG: Actual file hash (bytes): 59f4d3b66211bea5a5e7548bee5c1158bfaa04c8fc761be7ff237e0c83d5e029
Assinatura inválida para o arquivo 'documento.txt'. Os hashes não correspondem.

Adulteração detectada com sucesso (como esperado).
